# Clase 8 - Knn

**KNN: K-Nearest Neighbors o K vecinos más cercanos**

EL METODO SIRVE COMO PARA HACER PREDDCIONES COMO REGRESIONES, EL VALOR DE K DEPENDE DE LA EXPERIENCIA

Es un método de clasificación no paramétrico; es decir, no requiere asumir ninguna distribución para la variable aleatoria X=(X1,X2,...,Xp). Este método no requiere estimar las probabilidades desconocidas πg de que un elemento seleccionado al azar provenga de la población g.

La idea es buscar, para la nueva observación que queremos clasificar, sus K vecinos más cercanos, es decir, las K observaciones más cercanas respecto a una medida de distancia.

El algoritmo es el siguiente:

1. Definimos una medida de distancia adecuada para las observaciones.
2. Calculamos la distancia entre la nueva observación x0 que queremos clasificar, y las observaciones que tenemos en nuestra matriz de datos.
3. Seleccionamos las K observaciones más cercanas a x0, y miramos a qué grupo pertenecen.
4. Clasificamos x0 en la población a la que pertenece una mayor proporción de sus K vecinos.

## Carga de Librerias

In [6]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

#%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [12, 12]
np.random.seed(42)

# Vamos a usar este dataset para probar KNN en clasificación y en regresión
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import train_test_split

#F1 SOLO SIRVE PARA CLASIFICACIONES, (OBSERVAR QUE TAN BUEN CLASIFICADOR FUE)
from sklearn.metrics import f1_score

## Ejercicio

Vamos a ver como vamos a usar el algoritmo KNN en scikit-learn.
El algoritmo KNN se puede usar tanto en problemas de clasificación
(con el estimador KNeighborsClassifier) como en problemas de regresión(con el estimador KNeighborsRegressor)

**KMEDIAS ES UN METODO NO SUPERVISADO, ES DECIR QUE NO CONOCES LAS CLASIFICACIONES**

In [4]:
# Para este ejemplo vamos a usar el dataset CSM (Conventional and Social Media Movies) 
# que contiene información de la popularidad en redes sociales de distintas películas 
# así como las ventas en taquilla.
pelis = pd.read_csv("datos_peliculas.csv")
#SE QUITA EL TITULO PORQUE NO APORTA INFORMACION IMPORTANTE
pelis = pelis.drop("pelicula", axis=1) 
pelis.head()

,año,ratings,genero,ventas,presupuesto,secuela,vistas_youtube,positivos_youtube,negativos_youtube,comentarios,seguidores_agregados
0,2014,6.3,8,9130,4000000.0,1,3280543,4632,425,636,1120000.0
1,2014,7.1,1,192000000,50000000.0,2,583289,3465,61,186,12350000.0
2,2014,6.2,1,30700000,28000000.0,1,304861,328,34,47,483000.0
3,2014,6.3,1,106000000,110000000.0,2,452917,2429,132,590,568000.0
4,2014,4.7,8,17300000,3500000.0,2,3145573,12163,610,1082,1923800.0


In [8]:
#SE ELIGE COMO LA VARIABLE OBJETIVO EL GENERO
variable_objetivo_clasificacion = "genero"
#SE QUITA LA COLUMNA DE LA VARIABLE OBJETIVO PARA OBTENER LAS VARIABLES INDEPENDIENTES
variables_independientes_clasificacion = pelis.drop(variable_objetivo_clasificacion, axis=1).columns
#SE HACE LA PARTICION DE ENTRENAMIENTO Y PRUEBA 
X_train, X_test, y_train, y_test = train_test_split(
    pelis[variables_independientes_clasificacion],
    pelis[variable_objetivo_clasificacion], test_size=0.20, random_state=2021)

Los parámetros más importantes a la hora de usar KNeighborsClasifier son:
- **n_neighbors**: El valor de K, es decir el número de vecinos que considerar a la hora de asignar una clase.
- **weights**: A la hora de votar, que importancia dar a los vecinos. Si elegimos auto asigna la misma importancia a todos los vecinos. Si elegimos distance asigna importancia a los vecinos en función de la distancia de los vecinos al punto a clasificar
- **metric**: La métrica a la hora de medir la distancia entre los puntos. Si se usa distancia de Minkowsky se puede elegir p con el parámetro p, que por defecto es 2 (lo que computa la distancia euclidiana).

En este caso en particular sabemos que valor elegir de K, ya que podemos asumir  que el número de categorías del dataset es el total de categorías de películas del dataset de entrenamiento.

**Utilizando Weigths**

In [9]:
#COMO PUNTO DE PARTIDA SE TOMA LA LONGITUD DE LAS CATEGORIAS COMO PRIMER VALOR DE K
k_categorias = len(y_train.unique())
#SE APLICA EL CLASIFICADOR DANDO EL PESO UNIFORME
clasificador_knn = KNeighborsClassifier(n_neighbors=100, weights="uniform")
#SE LLENA CON LOS VALORES DE ENTRENAMIENTO
clasificador_knn.fit(X_train, y_train)
#SE HACEN LAS PREDICCIONES Y SE OBTINENE EL F1 SCORE
preds = clasificador_knn.predict(X_test)
print("f1 score para K clasificador utilizando peso uniforme: ")
print(f1_score(y_test, preds, average="micro"))

f1 score para K clasificador utilizando peso uniforme: 
0.40425531914893614


**Utilizando distance**

In [10]:
# Si ahora entrenamos el estimador con el argumento de pesos weights="distance", 
# vemos que funciona de forma ligeramente mejor.
clasificador_knn = KNeighborsClassifier(n_neighbors=100, weights="distance")
clasificador_knn.fit(X_train, y_train)
preds = clasificador_knn.predict(X_test)
print("f1 score para K clasificador utilizando peso de distancia: ")
print(f1_score(y_test, preds, average="micro"))

f1 score para K clasificador utilizando peso de distancia: 
0.3191489361702128


**NO PORQUE AUMENTES EL NUMERO DE VECINOS LA CLASIFICACION MEJORA**

Probando para diferentes valores de **K**

In [11]:
#SE UTILIZA UN CICLO PARA PROBAR LOS VALORES DE K
for k in range(1,51,2):
    #PARA UNIFORME
    clasificador_knn_unif = KNeighborsClassifier(n_neighbors=k, weights="uniform")
    clasificador_knn_unif.fit(X_train, y_train)
    preds_unif = clasificador_knn_unif.predict(X_test)
    
    #PARA DISTANCIAS
    clasificador_knn_distance = KNeighborsClassifier(n_neighbors=k, weights="distance")
    clasificador_knn_distance.fit(X_train, y_train)
    preds_distance = clasificador_knn_distance.predict(X_test)
    
    #SE IMPRIMEN LOS VALORES
    print(f'{(k,f1_score(y_test, preds_unif, average="micro"),f1_score(y_test, preds_distance, average="micro"))}')

#Podemos usar el método kneighbors para devolver los k vecinos de un punto en concreto


(1, 0.23404255319148937, 0.23404255319148937)
(3, 0.2978723404255319, 0.2553191489361702)
(5, 0.2765957446808511, 0.23404255319148937)
(7, 0.23404255319148937, 0.21276595744680848)
(9, 0.2553191489361702, 0.2553191489361702)
(11, 0.2765957446808511, 0.2553191489361702)
(13, 0.2978723404255319, 0.3191489361702128)
(15, 0.23404255319148937, 0.2978723404255319)
(17, 0.23404255319148937, 0.2765957446808511)
(19, 0.23404255319148937, 0.2553191489361702)
(21, 0.2765957446808511, 0.2553191489361702)
(23, 0.2553191489361702, 0.2765957446808511)
(25, 0.23404255319148937, 0.2765957446808511)
(27, 0.2765957446808511, 0.2978723404255319)
(29, 0.2765957446808511, 0.2978723404255319)
(31, 0.2765957446808511, 0.2765957446808511)
(33, 0.2978723404255319, 0.2978723404255319)
(35, 0.3191489361702128, 0.2978723404255319)
(37, 0.3404255319148936, 0.2978723404255319)
(39, 0.3191489361702128, 0.2765957446808511)
(41, 0.3404255319148936, 0.3191489361702128)
(43, 0.3404255319148936, 0.3191489361702128)
(45, 0

In [12]:
#SE IMPRIME EL PRIMER RENGLON DE LA TABLA ORIGINAL
print(X_test.iloc[0])

año                         2014.0
ratings                        5.2
ventas                  85900000.0
presupuesto             65000000.0
secuela                        1.0
vistas_youtube            930006.0
positivos_youtube           5150.0
negativos_youtube            707.0
comentarios                 1484.0
seguidores_agregados     5130800.0
Name: 13, dtype: float64


In [14]:
#AQUI SE OBTIENE EL VECINO MAS CERCANO DEL PRIMER RENGLON DE X_test
distancia, indice = clasificador_knn.kneighbors([X_test.iloc[0]], n_neighbors=1)
print(distancia,indice)

#EN DISTANCIA SE GUARDA LA DISTANCIA Y EL INDICE LLEVA EL INDICE DEL VECINO MAS CERCANO
print(X_train.iloc[indice[0]])

[[4622670.47679153]] [[129]]
     año  ratings    ventas  presupuesto  secuela  vistas_youtube  \
40  2014      7.6  85700000   68000000.0        1         2276605   

    positivos_youtube  negativos_youtube  comentarios  seguidores_agregados  
40               3946                331         1286             1888000.0  


c:\Users\felip\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


KNN para problemas de regresión

Vamos a utilizar ahora el algoritmo KNN para un problema de regresión,KNN funciona igual para hacer regresiones, simplemente que en vez de una votación donde la clase más común entre los vecinos más próximos es la elegida, se hace una interpolación de los valores de la variable numérica objetivo de los vecinos.

En concreto vamos a estimar las ventas de entradas en taquilla de una película en función de su popularidad online y presupuesto.

In [16]:
from sklearn.metrics import mean_squared_error

#SI DIVIDE EL DATA FRAME PARA EL OBJETIVO Y LAS VARIABLES INDEPENDIENTES
variable_objetivo_regresion = "ventas"
variables_independientes_regresion = pelis.drop(variable_objetivo_regresion, axis=1).columns

#SE DIVIDE EN ENTRENAMIENTO Y PRUEBA
X_train, X_test, y_train, y_test = train_test_split(
    pelis[variables_independientes_regresion],
    pelis[variable_objetivo_regresion], test_size=0.20)

# Usamos la implementación en sklearn KNeighborsRegressor para problemas de regresión.
# Tiene los mismos hiperparámetros que NeighborsClassifier.

#SE CREA EL ESTIMADOR DANDO EL NUMERO DE VECINOS Y EL PARAMETRO DE PESO
regresor_knn = KNeighborsRegressor(n_neighbors=10, weights="distance")

#SE LLENA EL ESTIMADOR CON LOS DATOS DE ENTRENAMIENTO
regresor_knn.fit(X_train, y_train)

#SE OBTIENEN LAS PREDICCIONES
preds = regresor_knn.predict(X_test)

#SE IMPRIME EL ERROR ENTRE LAS VARIABLES DE PRUEBA Y LAS PREDICCIONES
print(np.sqrt(np.abs(mean_squared_error(y_test, preds))))

43959789.048082694


In [17]:
#PARA TECNICAS DE CLASIFICICACION
#SE HACE LA PRUEBA DE VALIDACION CRUZADA

from sklearn.model_selection import cross_val_score

error_validacion_cruzada_clasificacion = np.sqrt(np.abs(
    cross_val_score(KNeighborsClassifier(
                n_neighbors=39, 
                weights="distance"), 
                X=pelis[variables_independientes_clasificacion],
                y=pelis[variable_objetivo_clasificacion], 
                scoring="f1_micro",
                cv = 20
        ).mean()
      )
)
print("La puntuación F1 de KNN para clasificacion en este dataset es {:.2f}".format(
    error_validacion_cruzada_clasificacion))

La puntuación F1 de KNN para clasificacion en este dataset es 0.57


c:\Users\felip\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=20.
  warnings.warn(
